In [ ]:
# Initialize OK
from client.api.notebook import Notebook
ok = Notebook('hw2.ok')

# Homework 2: Exploratory Data Analysis in Professional Basketball

In this assignment we'll conduct an exploratory data analysis of professional basketball data.  Basketball is a team sport in which the goal is to try to outscore the amount in a fixed amount of time. Points are scored (either 2 or 3 points) by putting the ball throw a hoop on one end of the court.  An attempt at putting the ball throw the hoop is known as a "shot".  If helpful, you can read more about [the rules of basketball](https://en.wikipedia.org/wiki/Rules_of_basketball).

The National Basketball Association (NBA) is the professional basketball league in the United States and provides a nice website with many statistics gathered on teams and players in the league: [http://stat.nba.com](http://stat.nba.com). 

## Part 1: Getting the Data

As an example, in order to navigate to the shooting records for Stephen Curry, one of the most famous players, you navigate their menus to get to here:

> [http://stats.nba.com/player/201939/shooting/?Season=2018-19&SeasonType=Regular%20Season](http://stats.nba.com/player/201939/shooting/?Season=2018-19&SeasonType=Regular%20Season)

Here, we see some information related to our choices:
- Season: 2018-19
- SeasonType: Regular Season (Note that [%20 is character code for space](https://en.wikipedia.org/wiki/Percent-encoding#Character_data))
- Player: 201939 (less obvious)

This type of URL is using a [GET method](https://www.w3schools.com/tags/ref_httpmethods.asp). When your URLs are very long, it is usually passing a series of variables and values to the web page. There are tools such as this [online URL parser](https://www.freeformatter.com/url-parser-query-string-splitter.html). Try passing in the URL.

Knowledge of how web sites work is useful for data science since there is so much interaction through the web.

We've provided a function that requests data from `stats.nba.com`. 

**User agent**: Inside the function there is a string variable for what is called a User Agent. Suitably setting this user agent string allows you to mimic any browser. Websites can return different content depending on the browser (i.e. user agent) users use to connect.  In the case of stats.nba.com, they block programatic scraping their pages using `wget` (with user agent, "Wget/1.19.4 (linux-gnu)" if you are using the course server) However, by specifying the user agent string in `get_nba_data`, we pretend that our connection is from a user using a Mozilla-type browser on OS X. 

**End point**: NBA has many different data: e.g., teams, players, games, etc. An end point is a URL designated for a specific type of data they provide. For example, a GET URL using `http://stats.nba.com/stats/commonallplayers` as the base URL will return players data. Other end points are listed here: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation.

Unfortunately, NBA seem have blocked the IP of the int15 cluster.  You can use the function on you own machine if python, [IPython](https://ipython.readthedocs.io/en/stable/interactive/python-ipython-diff.html#shell-assignment), and `wget` utility are installed.

The function will do the following:
1. Set User Agent
1. Set base URL with appropriate end point
1. Set parameters required for query
1. Read JSON string into python variable
1. Parse JSON string into python object
1. Convert the objects into pandas a data frame

In [1]:
def get_nba_data(endpt, params, return_url=False):

    ## endpt: https://github.com/seemethere/nba_py/wiki/stats.nba.com-Endpoint-Documentation
    ## params: dictionary of parameters: i.e., {'LeagueID':'00'}
    
    from pandas import DataFrame
    from urllib.parse import urlencode
    import json
    
    useragent = "\"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2) AppleWebKit/601.3.9 (KHTML, like Gecko) Version/9.0.2 Safari/601.3.9\""
    dataurl = "\"" + "http://stats.nba.com/stats/" + endpt + "?" + urlencode(params) + "\""
    
    # for debugging: just return the url
    if return_url:
        return(dataurl)
    
    jsonstr = !wget -q -O - --user-agent={useragent} {dataurl}
    
    data = json.loads(jsonstr[0])
    
    h = data['resultSets'][0]['headers']
    d = data['resultSets'][0]['rowSet']
    
    return(DataFrame(d, columns=h))

The function fetches the JSON and converts the JSON to a DataFrame that we can manipulate.  JSON format is very similar to Python dictionary: i.e., keys and values.  There are built-in libraries to work with json files formats. We read the output of `wget` command into a python variable: `json_str`. We parse that string with the `json` library.  First we load the necessary pacakges:

In [2]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import subprocess as sp
import pickle # to serialize/unserialize python data objects


The code below won't run on the int15 server.  If you have python installed on your local machine you can try scraping this data yourself!

In [3]:
## get all 2018-19 teams
if False:
    params = {'LeagueID':'00','Season': '2018-19'}
    teams = get_nba_data('commonTeamYears', params).set_index('TEAM_ID')
    allteams = teams.loc[teams.MAX_YEAR=='2018'].index.values

    pickle.dump(allteams, open( "allteams2018.p", "wb" ) )

In [4]:
# get all 2018-2019 player ids
if False:
    ## get all 2018-19 player ids
    params = {'LeagueID':'00', 'Season': '2018-19', 'IsOnlyCurrentSeason': '0'}
    players = get_nba_data('commonallplayers', params).set_index('PERSON_ID')

    # Player ids and names
    allplyrs = players.loc[players.TEAM_ID.isin(allteams)]['DISPLAY_FIRST_LAST'].drop_duplicates()

    ## save in a pickle file
    pickle.dump(allplyrs, open( "allplyrs2018.p", "wb" ) )

**Run the cell to get the files necessary files for this assignment.**

In [5]:
!wget -nc https://ucsb.box.com/shared/static/0shc0fcjslffepdfh9dc4unpxdvoumj0.json -O curry.json
!wget -nc https://ucsb.box.com/shared/static/qvslkv7l71g39jug1umc9hctkj87l96a.p -O allteams2018.p
!wget -nc https://ucsb.box.com/shared/static/ondn2cqs7l1s1f5fn5u11uzc0wbz0hfi.p -O allplyrs2018.p
!wget -nc https://ucsb.box.com/shared/static/8f2ify8ic0fzhntd5a5r2l1lessj41ky.p -O allshots2018.p

### Question 1a

Load the pickle titled `allplyrs2018.p` into a pandas dataframe called `allplyrs`:

<!--
BEGIN QUESTION
name: q1a
manual: false
points: 2
-->

In [6]:
allplyrs = ...

Below are the `params` we'll use for our `json` query.  For now we left `PlayerID` blank.  Most importantly we're querying data from 2018-19 season and looking at field goals: `'ContextMeasure':'FGA'`

In [7]:
params = {'PlayerID':'', # We'll fill this in after identifying the right player
          'PlayerPosition':'',
          'Season':'2018-19',
          'ContextMeasure':'FGA',
          'DateFrom':'',
          'DateTo':'',
          'GameID':'',
          'GameSegment':'',
          'LastNGames':'0',
          'LeagueID':'00',
          'Location':'',
          'Month':'0',
          'OpponentTeamID':'0',
          'Outcome':'',
          'Period':'0',
          'Position':'',
          'RookieYear':'',
          'SeasonSegment':'',
          'SeasonType':'Regular Season',
          'TeamID':'0',
          'VsConference':'',
          'VsDivision':''}
 

## Question 1b: Get one player's shot data

Use `allplyrs` to find the player id (index) associated with the player named "[Stephen Curry](https://en.wikipedia.org/wiki/Stephen_Curry)".  Set the value of `PlayerID` in the `params` dictionary to Stephen Curry's id.  

<!--
BEGIN QUESTION
name: q1b
manual: false
points: 2
-->

In [8]:
curry_id = ...

In [ ]:
ok.grade("q1b");

## Question 1c

The NBA stats page is blocking our jupyter hub server, so `get_nba_data` won't work on `int15.lsit.uscb` (if you have python installed on your machine, try it out!).  Instead, call the `get_nba_data` but have it return the url rather than actually query the website for the `json`.  Print the url.

In [11]:
if False:
    ## get shot data for Stephen Curry and pickle it
    curry_data = get_nba_data('shotchartdetail', params, return_url=False)
    pickle.dump(allplyrs, open( "curry2018.p", "wb" ) )
if True: 
    curry_url = ...
print(curry_url)

Copy `curry_url` and paste it into a new tab in your browser.  This will show you the raw `json` that is being served to us on a successful query.  We've saved this data as `curry.json` and included it for you in the `hw2` directory.  The code below will load the json file into a dictionary called `curry_json`.  The relevant data is under the key `resultSets`.  `h` will be the headers for this data and `d` will be the values in the table.  Use `h` and `d` to create a pandas data frame named `curry_data`.  Print the first few rows of `curry_data` to inspect it.

In [12]:
import json
with open('curry.json', 'r') as f:
    curry_json = json.load(f)
    
curry_json.keys()

h = curry_json['resultSets'][0]['headers']
d = curry_json['resultSets'][0]['rowSet']

curry_data = ...

## 1d Investigating the data about the data

A glossary of basketball terms can be found here: https://stats.nba.com/help/glossary/

Here are some of the key ones:
- Field goal: a basketball shot
- FGA: Field Goals Attempted 
- FGM: Field Goals Made


Group the data by `SHOT_ZONE_RANGE` and call the `size()` function on the resulting `groupby` data frame to get the number of shot attempts in different areas of the court.  Save this in `shot_zone_range_df`.  How many shots were taken within 8 feet of the basket? How many shots were taken further than 24 feet from the basket?  Save these in the appropriate variable below.

<!--
BEGIN QUESTION
name: q1d
manual: false
points: 2
-->

In [13]:
shot_zone_range_df = ...
shots_inside8 = ...
shots_24plus = ...

In [ ]:
ok.grade("q1d");

## 1e Investigating the data about the data


Lets try a different grouping now.  Group the data by `SHOT_ZONE_AREA` and and use `size()` to get the group counts.  Save this in `shot_zone_area_df` and print it.  How many shots were taken from the left center side or the right center side?

<!--
BEGIN QUESTION
name: q1e
manual: false
points: 2
-->

In [17]:
shot_zone_area_df = ...

*Write your answer here, replacing this text.*

### Question 1f 

Use pivot table to generate a table with entries representing the fraction of attempted shots that were made (i.e. compute the field goal percentage) by both range and area for each period of the game. Set the rows (`index`) to correspond to the `PERIOD` and the columns corresponding to `SHOT_ZONE_RANGE` and `SHOT_ZONE AREA`.  Fill any missing entries in the pivot table by setting an [appropriate argument](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html).


<!--
BEGIN QUESTION
name: q1f
manual: false
points: 2
-->

In [18]:
FGpercent_table = ...


In [ ]:
ok.grade("q1f");

### Question 1e

You are interested in identifying whether Stephen Curry has different behavior in different periods of the game.  In a few sentences, discuss how might you identify whether Curry's shot selection is different early in the game (e.g. periods 1 and 2) than later in the game (e.g. periods 3 and 4)?  This is an open ended question and the answer can include looking at visualizations, statistical tests etc.

*Write your answer here, replacing this text.*

## Part 2: Visualization

### Q2a
Use `seaborn` to create scatter plot of the location of Stephen Curry's shot attempts from this year (`LOC_X` and `LOC_Y`).  When you call a scatterplot, seaborn returns a figure in an object, we'll call it `ax`.  We can set properties of the figure by calling methods on `ax`.  Use this approach to set the x-axis limits to span (-300, 300), the y-axis limits to span (-100, -500).

<!--
BEGIN QUESTION
name: q2a
manual: false
points: 2
-->

In [20]:
import seaborn as sns

plt.figure(figsize=[12, 11])
ax = ...

...


In [ ]:
ok.grade("q2a");


Understanding any dataset is hard without context.  Lets add some important context by adding the relevant court lines into our diagram.  If you are interested, you can read more about the lines and dimensions on the [NBA basketball court](https://en.wikipedia.org/wiki/Basketball_court).  We will use code from [http://savvastjortjoglou.com/nba-shot-sharts.html](http://savvastjortjoglou.com/nba-shot-sharts.html) to add the court markings to our diagram.  The `draw_court` function below will do this for us.  The below cell will generate an example court.


In [23]:
## code is from http://savvastjortjoglou.com/nba-shot-sharts.html
def draw_court(ax=None, color='black', lw=1, outer_lines=False):
    
    from matplotlib.patches import Circle, Rectangle, Arc
    from matplotlib.pyplot import gca
    
    # If an axes object isn't provided to plot onto, just get current one
    if ax is None:
        ax = gca()

    # Create the various parts of an NBA basketball court

    # Create the basketball hoop
    # Diameter of a hoop is 18" so it has a radius of 9", which is a value
    # 7.5 in our coordinate system
    hoop = Circle((0, 0), radius=7.5, linewidth=lw, color=color, fill=False)

    # Create backboard
    backboard = Rectangle((-30, -7.5), 60, 0, linewidth=lw, color=color)

    # The paint
    # Create the outer box 0f the paint, width=16ft, height=19ft
    outer_box = Rectangle((-80, -47.5), 160, 190, linewidth=lw, color=color,
                          fill=False)
    # Create the inner box of the paint, widt=12ft, height=19ft
    inner_box = Rectangle((-60, -47.5), 120, 190, linewidth=lw, color=color,
                          fill=False)

    # Create free throw top arc
    top_free_throw = Arc((0, 142.5), 120, 120, theta1=0, theta2=180,
                         linewidth=lw, color=color, fill=False)
    # Create free throw bottom arc
    bottom_free_throw = Arc((0, 142.5), 120, 120, theta1=180, theta2=0,
                            linewidth=lw, color=color, linestyle='dashed')
    # Restricted Zone, it is an arc with 4ft radius from center of the hoop
    restricted = Arc((0, 0), 80, 80, theta1=0, theta2=180, linewidth=lw,
                     color=color)

    # Three point line
    # Create the side 3pt lines, they are 14ft long before they begin to arc
    corner_three_a = Rectangle((-219, -47.5), 0, 140, linewidth=lw,
                               color=color)
    corner_three_b = Rectangle((219, -47.5), 0, 140, linewidth=lw, color=color)
    # 3pt arc - center of arc will be the hoop, arc is 23'9" away from hoop
    # I just played around with the theta values until they lined up with the 
    # threes
    three_arc = Arc((0, 0), 475, 475, theta1=22.5, theta2=157.5, linewidth=lw,
                    color=color)

    # Center Court
    center_outer_arc = Arc((0, 422.5), 120, 120, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    center_inner_arc = Arc((0, 422.5), 40, 40, theta1=180, theta2=0,
                           linewidth=lw, color=color)

    # List of the court elements to be plotted onto the axes
    court_elements = [hoop, backboard, outer_box, inner_box, top_free_throw,
                      bottom_free_throw, restricted, corner_three_a,
                      corner_three_b, three_arc, center_outer_arc,
                      center_inner_arc]

    if outer_lines:
        # Draw the half court line, baseline and side out bound lines
        outer_lines = Rectangle((-250, -47.5), 500, 470, linewidth=lw,
                                color=color, fill=False)
        court_elements.append(outer_lines)

    # Add the court elements onto the axes
    for element in court_elements:
        ax.add_patch(element)

    return ax

plt.figure(figsize=(12,11))
draw_court(outer_lines=True)
plt.xlim(-300,300)
plt.ylim(-100,500)
plt.show()

### Q2b

Again use seaborn to make a scatter plot of Stephen Curry's shots. Again, set the x-axis limits to span (-300, 300), the y-axis limits to span (-100, -500) color the points by whether the shot was made or missed.  Set the missed shots to have an 'x' symbol and made shots to be a circular symbol. Call the `draw_court` function with `outer_lines` set to to be true. Save the `Axes` returned by the plot call in a variable called `ax`.

<!--
BEGIN QUESTION
name: q2b
manual: false
points: 2
-->

In [24]:
plt.figure(figsize=(12, 11))
#cmap = sns.cubehelix_palette(dark=0, light=1, as_cmap=True)
markers = {0 : "X", 1 : "o"}
ax = ...

...



In [ ]:
ok.grade("q2b");

### Q2c
In a few sentences, discuss what makes this an effective or ineffective visualization for understanding the types of shots that Stephen Curry likes to take and is good at taking, relative to other players in the league.  Are there ways it can be improved?

*Write your answer here, replacing this text.*

### Question 2d: A Hexbin plot

Visualize Stephen Curry's shots by using a [hexbin plot with marginal histograms](https://seaborn.pydata.org/examples/hexbin_marginals.html). Also refer to setting [figure aesthetics](https://seaborn.pydata.org/tutorial/aesthetics.html) for what commands below do.

In [27]:
sns.set_style("white")
joint_shot_chart = sns.jointplot(x="LOC_X", y="LOC_Y", data=curry_data, kind='hex', space=0)
joint_shot_chart.fig.set_size_inches(12,11)

# A joint plot has 3 Axes, the first one called ax_joint 
# is the one we want to draw our court onto and adjust some other settings
ax = joint_shot_chart.ax_joint
draw_court(ax, outer_lines=True)

# Adjust the axis limits and orientation of the plot in order
# to plot half court, with the hoop by the top of the plot
ax.set_xlim(-300, 300)
ax.set_ylim(500, -100)

# Get rid of axis labels and tick marks
ax.set_xlabel('')
ax.set_ylabel('')
ax.tick_params(labelbottom=False, labelleft=False)

# Add a title
ax.set_title('Stephen Curry, 2018-19, FGA', 
             y=1.2, fontsize=18)

# Add Data Scource and Author
ax.text(-250,445,'\n This plot is based on code by Savvas Tjortjoglou (savvastjortjoglou.com)',
        fontsize=12);

## Question 3: Analyzing Shots from All Players


As a reminder, we've already gathered a list of all player names and ids in the `allplyrs` object.  The code below in the `if False` look will not run on `int15.lsit.ucsb.edu`, but if you run it locally, it will loops over all player ids and grabs the `shotchartdetail` from [stats.nba.com](https://stats.nba.com). We've done this step and stored the pandas data frame of all shot data in `allshots2018.p`. Load this pickle file into a DataFrame titled `allshots`.


In [28]:
if False:
    params = {'PlayerID':'',
              'PlayerPosition':'',
              'Season':'2018-19',
              'ContextMeasure':'FGA',
              'DateFrom':'',
              'DateTo':'',
              'GameID':'',
              'GameSegment':'',
              'LastNGames':'0',
              'LeagueID':'00',
              'Location':'',
              'Month':'0',
              'OpponentTeamID':'0',
              'Outcome':'',
              'Period':'0',
              'Position':'',
              'RookieYear':'',
              'SeasonSegment':'',
              'SeasonType':'Regular Season',
              'TeamID':'0',
              'VsConference':'',
              'VsDivision':''}

    shotdf = []
    for p in allplyrs.index.values:
        print(p)

        # get player p's data
        params['PlayerID'] = p
        
        # scrape the json data
        playerdata = get_nba_data('shotchartdetail', params)
        
        # add playerID to the dictionary
        playerdata['PlayerID'] = p
        
        # add to the list of player data frames
        shotdf += [playerdata]
    
    # This concats a list of data frame's into one big data frame
    allshots = pd.concat(shotdf)

    # Save the json
    pickle.dump(allshots, open('allshots2018.p', 'wb'))
    
if True:
    
    allshots = ...

### Q3a

From all shots taken in 2018-2019, create a DataFrame `shot_counts` of shots taken in each zone range and area combination. Then, calculate the fraction of shots taken from range `24+ ft.` in the `Center(C)` shot area.

<!--
BEGIN QUESTION
name: q3a
manual: false
points: 2
-->

In [29]:
shot_counts = ...
...

frac_center24 = ...

In [ ]:
ok.grade("q3a");

### Question 3b

Make a histogram of the number of total shots taken across all players **with 15 bins**.  Label the x-axis appropriately, set the color of the bars to green.

<!--
BEGIN QUESTION
name: q3b
manual: false
points: 2
-->

In [31]:
plt.figure(figsize=(8, 7))
ax = ...
...


## Q3c

What do you observe about the distribution of shot attempts in the NBA across all players? Why might this be? Discuss in 1-3 sentences below.

*Write your answer here, replacing this text.*

## Question 4: Binning and Smoothing Shots

So far, in we have worked with dataframes which represent each shot as a single observation (row) within the dataset.  However, this isn't a convenient data structure for the kinds of spatial analyses we will puruse below.  In this part, we will divide the court into square regions and create a matrix which includes the number of shots taken by a player in that region.  We divide the court up into square bins (e.g. a 2d histogram), and, for each player, count number of shots that fall into each bin. Fortunately, this function is relatively simple to write using existing `numpy` functions. 

### Q4a


Fill in the `bin_shots` function below. Use `np.histgram2d` to count count the shots in each bin.  The bins are defined `bin_edges` which is a pandas Series of the form `(xedges, yedges)`.  If `density = True`, call `ndimage.filters.gaussian_filter` on the result of `np.histogram2d` with smoothing parameter `sigma`.  This will create a smoothed version of the raw data histograms.

<!--
BEGIN QUESTION
name: q4a
manual: false
points: 2
-->

In [32]:
def bin_shots(df, bin_edges, density=False, sigma=1):
    
    """Given data frame of shots, compute a 2d matrix of binned counts is computed
    
    Args:
        df: data frame of shotchartdetail from nba.com. 
            At the minimum, variables named LOCX and LOCY are required.
        bin_edges: bin edge definition: edges in x and edges in y
    
    Returns:
        binned: counts
        xedges: bin edges in X direction
        yedges: bin edges in Y direction
    """
    import numpy as np
    from scipy import ndimage
    
    ## Call np.histogram2d
    ...
    
    if density:

        # Recompute 'binned' using "gaussian_filter"
        binned = ...
        
        # Normalize the histogram to be a "density", e.g. mass across all bins sums to 1.
        binned /= np.sum(binned)
    
    return(binned, xedges, yedges)


In [ ]:
ok.grade("q4a");

### Q4b: Visualize the binning on `curry_data`

Call `bin_shots` on `curry_data` to create a binned but unsmoothed matrix of shot counts (call this `curry_binned_unsmoothed`), a binned and smoothed matrix of counts with `sigma=1` (call this `curry_binned_smoothed1`) and one with `sigma=5` (call this `curry_binned_smoothed5`). Use the bin edges defined below:

In [37]:
## bin edge definitions in inches
xedges = np.linspace(start=-300, stop=300, num=151)
yedges = np.linspace(start=-48, stop=372, num=106)

In [38]:
bin_edges = (xedges, yedges)
...
...
...

The function below can be used to visualize the shots as a heatmap:

In [39]:
def plot_shotchart(binned_counts, xedges, yedges, ax=None, use_log=False, cmap = 'Reds'):
    
    """Plots 2d heatmap from vectorized heatmap counts
    
    Args:
        hist_counts: vectorized output of numpy.histogram2d
        xedges, yedges: bin edges in arrays
        ax: figure axes [None]
        use_log: will convert count x to log(x+1) to increase visibility [False]
        cmap: Set the color map https://matplotlib.org/examples/color/colormaps_reference.html
    Returns:
        ax: axes with plot
    """
    
    import numpy as np
    import matplotlib.pyplot as plt
    
    ## number of x and y bins.  
    nx = xedges.size - 1
    ny = yedges.size - 1

    X, Y = np.meshgrid(xedges, yedges)
    
    if use_log:
        counts = np.log(binned_counts + 1)
        
    if ax is None:
        fig, ax = plt.subplots(1,1)
    
    ax.pcolormesh(X, Y, binned_counts.T, cmap=cmap) 
    ax.set_aspect('equal')
    
    draw_court(ax)
    
    return(ax)



Create 3 side by side plots of `curry_binned_unsmoothed`, `curry_binned_smoothed1` and `curry_binned_smoothed5`

<!--
BEGIN QUESTION
name: q4b
manual: false
points: 2
-->

In [40]:
fig, ax = plt.subplots(1, 3, figsize=(20,60))

...

### 4c Vectorizing Shot Images

- Here we proceed create a dictionary of smoothed patterns, each vectorized into a 1-d array. 
- In this case, the object `all_smooth` is a dictionary that consists of arrays of length `15750`. 
- Each entry in `all_smooth` represents the smoothed frequency of shots along the bins generated in the code above for a given player.

In [41]:
## number of bins is one less than number of edges (remember homework 1)
nx = xedges.size - 1
ny = yedges.size - 1

## 2d histogram containers for binned counts and smoothed binned counts
all_counts = []
all_smooth = []
pids = []

## 2d histogram containers for binned counts and smoothed binned counts

## data matrix: players (row) by vectorized 2-d court locations (column)
for i, one in enumerate(allshots.groupby('PlayerID')):
    
    ## what does this line do?
    pid, pdf = one
        
    num_shots = len(pdf.index)
    if(num_shots > 100): 
        
        tmp1, xedges, yedges = bin_shots(pdf, bin_edges=(xedges, yedges), density=True, sigma=2)
        tmp2, xedges, yedges = bin_shots(pdf, bin_edges=(xedges, yedges), density=False)
    
        ## vectorize and store into list
        all_smooth += [tmp1.reshape(-1)]
        all_counts += [tmp2.reshape(-1)]
        pids += [pid]

In [42]:
X = np.vstack(all_smooth)
n, p = X.shape

print('Number of shot regions (p):', p)
print('Number of players (n):', n)

## Question 5: Non-negative Matrix Factorization (NMF) 

The non-negative matrix factorization is a dimension reduction technique that is often applied to image data.  It is similar to PCA except that is only applicable for strictly positive data.  We can apply the NMF to vectorized versions of the shot surface. This is useful because we can conver the observed matrix of shot surfaces into:
* Bases: Identifying modes of shooting style (number of modes is determined by `n_components` argument to `NMF` function below)
* Coefficients: How each players shooting style could be expressed as a (positive) linear combination of these bases

The NMF solves the following problem: given some matrix $X$ is $n \times p$ matrix ($n$ players, $p$ court regions), NMF computes the following factorization:
$$ \min_{W, H} \| X - HW^T \|_F\\
\text{ subject to } W\geq 0,\ H\geq 0, $$
where $W$ is ${p \times r}$ matrix and $H$ is ${n\times r}$ matrix.

In this homework, we have the following:
 
#### The data matrix $X$ 
$X$ is of dimension $n$={number of players} and $p$={number of total square bins on the court}. Each row corresponds to a player, with observations corresponding to a "flattened" or "vectorized" version of the 2d histograms plotted in part 4b.  In other words, the rows of $X$ contain the counts of shots in each rectangular region of the court, for a player.

#### Bases matrix: $W$

Columns, $W_i$, contain the shot "bases". First, we will try it with $r=3$ bins in 5a, and then with $r=10$ bins in 5d.

#### Coefficient matrix: H

Each row of $H$ gives a coefficient (or "weight") for each of the bases vectors in $W$, and there are $n$ rows, one for each player.


The `sklearn` library is one of the main Python machine learning libraries.  It has a built in NMF function for us.  The function below runs this function and normalizes the basis surfaces to sum to 1.

In [43]:
## Non-negative Matrix Factorization
def non_negative_marix_decomp(n_components, array_data):
    import sklearn.decomposition as skld
    
    ## define the NMF model we will use
    model = skld.NMF(n_components=n_components, init='nndsvda', max_iter=500, random_state=0)
    
    ## fit the transform to get W
    W = model.fit_transform(array_data.T)
    
    # Normalize basis vectors to sum to 1
    Wsum = W.sum(axis=0)
    W = (W/Wsum)
    
    ## adjust H correspondingly
    H = model.components_
    H = (H.T * Wsum)

    nmf = (W, H)
    return(nmf)

### Q5a

Compute the NMF on all player's shot charts, X, assuming with `n_components` = 3 (i.e. each shot chart can be represented as a positive linear combination of 3 "basis" shot charts).  Fill in `plot_vectorized_shot_chart`.  This takes a the a vector of binned shot counts, converts it back to a matrix of the appropriate size and then calls `plot_shotchart` on the matrix.  The numpy function `reshape` will be useful here: [https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html)

In [44]:
...

In [45]:
print(X.shape)
print(W3.shape)
print(H3.shape)

In [46]:
def plot_vectorized_shotchart(vec_counts, xedges, yedges, ax=None, use_log=False, cmap = 'Reds'):
    
    """Plots 2d heatmap from vectorized heatmap counts
    
    Args:
        hist_counts: vectorized output of numpy.histogram2d
        xedges, yedges: bin edges in arrays
        ax: figure axes [None]
        use_log: will convert count x to log(x+1) to increase visibility [False]
        cmap: Set the color map https://matplotlib.org/examples/color/colormaps_reference.html
    Returns:
        ax: axes with plot
    """

    nx = ...
    ny = ...
    
    # use reshape to convert a vectorized counts back into a 2d histogram
    two_d_counts = ...
    
    return(plot_shotchart(two_d_counts, xedges, yedges, ax=ax, use_log=use_log, cmap=cmap))



### Q5b

Plot the first three basis images by calling `plot_vectorized_shot_chart` on the columns of W3.

In [47]:

fig, ax = plt.subplots(1, 3, figsize=(20,60))

## Write a for loop
for i in range(3):
    # Call plot_vectorized_shot_chart
    ax[i].set_title('Shot Basis %i' % (i+1))

### Q5b
Below we re-construct the shooting pattern for a single player. By "reconstructing" we mean use the approximation $$\hat{X} = HW^T$$ obtained via NMF.  Find $\hat X$ by multipling H and $W^T$.  In python the `@` symbol is used for matrix multiplication.

In [48]:
X3_hat = ...


Plot $X$, $\hat X$ and the residual ($X - \hat X$) for the player named LaMarcus Aldridge. Remember, each *row* of $X$ is a vectorized matrix corresponding to the binned (or smoothed binned) shot information.

In [49]:
# Find the player_id of LaMarcus Aldridge
player_id = ...

## find index in X corresponding to that player
to_plot_idx = np.where(pids == player_id)[0][0]

fig, ax = plt.subplots(1, 3, figsize=(20,60))

## Call plot_vectorized_shot_chart
original_shotchart = ...
reconstructed_shotchart = ...
residual_chart = ...

print(max(abs(X3_hat[:, to_plot_idx] - X[:, to_plot_idx])))
ax[0].set_title('Original Shooting Pattern')
ax[1].set_title('Reconstructed Shooting pattern (r=3)')
ax[2].set_title('Residual Shooting Pattern (r=3)')

### Q5c

Why does it make sense to use a _sequential_ palette for the original and reconstructed shot charts and a _diverging_ palette for the residual?  _Hint:_ Read the introduction to colormaps [here](https://matplotlib.org/users/colormaps.html). 

*Write your answer here, replacing this text.*

What areas of the court does this player to shoot more and where less relative to the reconstructed area.  If its helpful, you can refer to court locations by name using this legend [here](https://en.wikipedia.org/wiki/Basketball_court#/media/File:Basketball_terms.png]).

*Write your answer here, replacing this text.*

#### Q5d

Re-run the analysis, this time for 10 basis vectors instead of 3.  Again plot the bases using `plot_vectorized_shotchart` on the columns of `W10`.

In [50]:
...

fig, ax = plt.subplots(2, 5, figsize=(20, 7))

## Write a for loop
for i in range(10):
    ax[i//5, i % 5].set_title('Shot Basis %i' % (i+1))


If you did things correctly, you should be really impressed! We've identified potentially interesting patterns of shooting styles without actually specifying anything about the way basketball is played or where the relevant lines are on the court.  The resulting images are based only on the actual behavior of the players.  Even more impressive is that we're capturing similarity in regions that are far apart on the court.  One reason we can do this is that a basketball court is symmetric along the length of the court (i.e. symmetric about x=0).  However, people tend to be left or right hand dominant, which might affect their preferences.  Look carefuly at the shot basis plots above: is there any evidence of _asymmetry_ in player shooting behavior?  Refer to specific basis images in your answer.    

*Write your answer here, replacing this text.*

Repeat part 5b, and again plot original, reconstructed and residual shot chats for LaMarcus Aldridge.

In [51]:
X10_hat = ...

fig, ax = plt.subplots(1, 3, figsize=(20,60))

# I took the first player appearing in first column 
# (you probably want to do more interesting players)
original_shotchart = ...
reconstructed_shotchart = ...
residual_chart = ...

ax[0].set_title('Original Shooting Pattern')
ax[1].set_title('Reconstructed Shooting pattern (r=10)')
ax[2].set_title('Residual Shooting Pattern (r=10)')

### Q5g: Next steps

One of the main purposes of exploratory data analysis is to generate new ideas, directions, and hypothesis for future analyses and experiments.  What did you see in this analysis that piqued your interest? What are some directions for follow up analyses that you could do based on what you found in this analysis? What types of tools (programming, statistical or otherwise) would you need for this analysis.  What additional data might you need? Discuss in a paragraph below.

*Write your answer here, replacing this text.*

# Submit
Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output.
**Please save before submitting!**

In [ ]:
# Save your notebook first, then run this cell to submit.
ok.submit()